In [1]:
from functools import partial

import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.common import get_top_left_x
from video699.screen.semantic_segmentation.evaluation import evaluate, all_frames_visualization, all_video_statistics
from video699.screen.semantic_segmentation.fastai_detector import FastAIScreenDetector, FastAIScreenDetectorVideoScreen, ALL_VIDEOS
from video699.screen.semantic_segmentation.postprocessing import approximate
from video699.video.annotated import AnnotatedSampledVideoScreenDetector

In [29]:
detector = FastAIScreenDetector()

In [30]:
method_params = list(detector.methods.keys())
train_params = list(detector.train_params.keys())
all_params = train_params + method_params

In [31]:
base_lower_bounds = [5, 7]
base_upper_bounds = [40, 70]
base_factors = [[0.1, 0.01]]

erode_dilate_lower_bounds = [5, 7, 15]
erode_dilate_upper_bounds = [40, 50]
erode_dilate_factors = [[0.1, 0.01]]
erode_dilate_iterations = [40, 100]

ratio_split_lower_bounds = [0.7, 0.9]
ratio_split_upper_bounds = [1.1, 1.5]

methods_values = [base_lower_bounds] + [base_upper_bounds] + [base_factors] + [erode_dilate_lower_bounds] \
        + [erode_dilate_upper_bounds] + [erode_dilate_factors] + [erode_dilate_iterations] \
        + [ratio_split_lower_bounds] + [ratio_split_upper_bounds]

In [32]:
batch_size = [8]
resize_factor = [2]
frozen_epochs = [7]
unfrozen_epochs = [6]
frozen_lr = [1e-2, 1e-3, 1e-4]
unfrozen_lr = [[1e-4, 2e-4], [1e-3, 1e-4]]

train_params_values = [batch_size] + [resize_factor] + [frozen_epochs] + [unfrozen_epochs] \
                    +[frozen_lr] + [unfrozen_lr]

In [33]:
method_settings = list(product(*methods_values))
train_settings = list(product(*train_params_values))

In [86]:
train_params_dict = dict(zip(train_params, train_settings[0]))

In [87]:
all_lectures = [video.filename for video in ALL_VIDEOS]
valid_lectures = all_lectures[:5]
train_lectures = all_lectures[5:10]

In [92]:
train_params_dict['frozen_epochs'] = 1
train_params_dict['unfrozen_epochs'] = 1
train_params_dict['resize_factor'] = 4
train_params_dict['frozen_lr'] = [1e-03, 1e-04]

In [93]:
filtered_by = lambda name: any([lecture in str(name) for lecture in train_lectures + valid_lectures]) and 'frame' in str(name)
split_by = lambda name: any([lecture in str(name) for lecture in valid_lectures])
detector = FastAIScreenDetector(train_params=train_params_dict, filtered_by = filtered_by, valid_func=split_by)

In [94]:
detector.train_params

{'batch_size': 8,
 'resize_factor': 4,
 'frozen_epochs': 1,
 'unfrozen_epochs': 1,
 'frozen_lr': [0.001, 0.0001],
 'unfrozen_lr': [0.0001, 0.0002]}

In [95]:
detector.train()

AssertionError: List len mismatch (2 vs 3)

In [ ]:
detector.learner.data.add_test(SegmentationCustomList.from_folder()
preds = detector.get_preds(DatasetType.Test)[0].squeeze().numpy()

In [ ]:
tensor = cv_image_to_tensor(frame.image)
tensor = self.learner.predict(tensor)
pred = tensor_to_cv_binary_image(tensor)
resized = resize_pred(pred, tuple(self.src_shape))